In [35]:
import os 
import numpy as np
import pandas as pd

original_train_dir = os.path.abspath("./original_data/train/train")
train_pictures = os.listdir(original_train_dir)

df = pd.DataFrame(train_pictures, columns = ['filename'])
df['label'] = df.filename.str[:3]
df
    
#np.random.randint()

,filename,label
0,dog.8011.jpg,dog
1,cat.5077.jpg,cat
2,dog.7322.jpg,dog
3,cat.2718.jpg,cat
4,cat.10151.jpg,cat
...,...,...
24995,dog.8008.jpg,dog
24996,dog.1992.jpg,dog
24997,dog.12412.jpg,dog
24998,cat.2701.jpg,cat
